## Project of Segmenting and Clustering Neighborhoods in Toronto Part 2

## Libraries

In [32]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## Downloading and Preparing Data

In [33]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup_obj = BeautifulSoup(url.text, 'lxml')

data = []
columns = []
table = soup_obj.find(class_='wikitable')

for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    if (index == 0):
        columns = section
    else:
        data.append(section)

#DataFrame
df = pd.DataFrame(data = data,columns = columns)
df = df[df['Borough'] != 'Not assigned']
df["Neighborhood"] = df.groupby("Postal code")["Neighborhood"].transform(lambda n_group: ', '.join(n_group))

df = df.drop_duplicates()

if(df.index.name != 'Postal code'):
    df = df.set_index('Postal code')
    
df['Neighborhood'].replace("Not assigned", df["Borough"],inplace=True)
df.head()


,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [34]:

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [35]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [36]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
geo_df.rename(columns={'Postal Code':'Postal code'},inplace=True)
merged_df = pd.merge(geo_df, df, on='Postal code')

In [38]:
toronto_df=merged_df[['Postal code','Borough','Neighborhood','Latitude','Longitude']]

In [39]:
toronto_df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
